In [1]:
import numpy as np
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, Embedding, Dropout, Bidirectional
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Parameters
vocab_size = 10000
max_seq_length = 100  # Maximum length of sequences
embedding_dim = 150   # Size of word embeddings
latent_dim = 128     # Size of the latent space
rnn_cells_first = 256  # Number of cells in the first RNN layer
rnn_cells_second = 128  # Number of cells in the second RNN layer
epochs = 100
batch_size = 256


In [2]:
# Load Reuters dataset
(x_train, _), (x_test, _) = reuters.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=max_seq_length, padding='post')
x_test = pad_sequences(x_test, maxlen=max_seq_length, padding='post')

2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Encoder
input_text = Input(shape=(max_seq_length,))
x = Embedding(vocab_size,embedding_dim)(input_text)  # Embedding layer
x = Bidirectional(LSTM(rnn_cells_first,return_sequences=True))(x)  # Apply Bidirectional LSTM layer, remember to return all steps of the sequence
x = Dropout(0.15)(x)  # Apply dropout regularization
x = Bidirectional(LSTM(rnn_cells_second,activation='relu'))(x)  # Apply another Bidirectional LSTM
latent_repr = Dense(latent_dim, activation='relu')(x)  # Dense layer with ReLU activation
encoder_model = Model(input_text, latent_repr)  # Define encoder model
encoder_model.summary()  # Display model summary

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 150)       │     1,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 512)       │       833,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       656,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,022,816 (11.53 MB)

 Trainable params: 3,022,816 (11.53 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
decoder_input = Input(shape=(latent_dim,))  # (None, latent_dim)
decoder_x = Dense(embedding_dim, activation='relu')(decoder_input)  # (None, embedding_dim)
decoder_x = RepeatVector(max_seq_length)(decoder_x)  # (None, max_seq_length, embedding_dim)
decoder_x = LSTM(rnn_cells_second, return_sequences=True)(decoder_x)
decoder_x = Dropout(0.15)(decoder_x)  # Fixed: use decoder_x, not x
decoder_x = LSTM(rnn_cells_first, return_sequences=True)(decoder_x)  # Fixed: return_sequences=True (not string)
decoder_output = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_x)

decoder_model = Model(decoder_input, decoder_output)
decoder_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        19,350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 100, 150)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100, 128)       │       142,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100, 256)       │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 10000)     │     2,570,000 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,126,438 (11.93 MB)

 Trainable params: 3,126,438 (11.93 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:

# Autoencoder
autoencoder = Model(input_text, decoder_model(encoder_model(input_text)))
autoencoder.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy')
autoencoder.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional (Functional)         │ (None, 128)            │     3,022,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 100, 10000)     │     3,126,438 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,149,254 (23.46 MB)

 Trainable params: 6,149,254 (23.46 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:

from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

def custom_lr_scheduler(epoch, lr):
    # Decrease learning rate by 0.1 factor every 5 epochs
    if epoch%5==0:
      lr=lr+0.1
    return lr

# Define the callback
lr_scheduler = LearningRateScheduler(custom_lr_scheduler)

early_stopping = EarlyStopping(monitor='loss',patience=5,restore_best_weights=True)  # Define EarlyStopping on val_loss

# Train the model
autoencoder.fit(x_train,x_train,epochs=epochs,batch_size=batch_size,validation_data=[x_test,x_test],callbacks=[lr_scheduler,early_stopping])


Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 45s 795ms/step - loss: nan - val_loss: nan - learning_rate: 0.1005
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 742ms/step - loss: nan - val_loss: nan - learning_rate: 0.1005
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 741ms/step - loss: nan - val_loss: nan - learning_rate: 0.1005
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 41s 735ms/step - loss: nan - val_loss: nan - learning_rate: 0.1005
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 41s 725ms/step - loss: nan - val_loss: nan - learning_rate: 0.1005
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 26s 724ms/step - loss: nan - val_loss: nan - learning_rate: 0.2005


In [7]:
# Encode the input text using the encoder model
encoded_texts = encoder_model.predict(x_test[:10])

# Decode the encoded texts using the decoder model
# This gives you a probability distribution for each word in the sequence
prob_distributions = decoder_model.predict(encoded_texts)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 522ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 999ms/step


In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')

    preds = np.clip(preds, 1e-7, 1)  # Avoid log(0)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)

    preds = exp_preds / np.sum(exp_preds)  # Normalize

    preds = np.nan_to_num(preds)  # Replace NaN with 0

    preds /= np.sum(preds)  # ✅ FORCE sum to exactly 1

    return np.random.choice(len(preds), p=preds)


In [15]:
def decode_sequence_with_sampling(prob_distributions, temperature=1.0):
    words = []
    for probs in prob_distributions:
        probs = np.asarray(probs)

        # Add this debug print
        if np.any(np.isnan(probs)) or np.any(probs < 0):
            print("Bad probs detected:", probs)
            break

        sampled_word = sample(probs, temperature)
        word = reverse_word_index.get(sampled_word - 3, '?')
        words.append(word)
    return ' '.join(words)


In [16]:

# Example usage with a specified temperature
temperature = 1.5  # Adjust this value to change randomness
decoded_texts = [decode_sequence_with_sampling(seq, temperature) for seq in prob_distributions]
for text in decoded_texts:
    print('\n')
    print(f'Decoded: {text}')
    print('\n')


Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]
Bad probs detected: [nan nan nan ... nan nan nan]


Decoded: 




Decoded: 




Decoded: 




Decoded: 




Decoded: 




Decoded: 




Decoded: 




Decoded: 




Decoded: 




Decoded: 


